In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from tickers import getTickerList, getCandlePatternList
from datemodule import getDateRange
from indicators import ChannelBreakoutIndicator, SupportResistanceIndicator
from datasource import getFullData
import pandas

In [8]:
import talib
import pandas_ta as ta
from tqdm import tqdm

tickers = getTickerList(all=True)
tickerShortlist = {}
tickerShortlistData = {}

for i in tqdm(range(len(tickers))):
    ticker = tickers[i]
    resultDict = {}
    tickerShortlistData[ticker] = {}

    # Fetch Data
    startDate, endDate = getDateRange('1y')
    df = getFullData(ticker, startDate, endDate)
    
    candleIndex = len(df)-1
    
    # RSI
    df['RSI'] = ta.rsi(df.Close, length=14)

    # Candlestick Pattern Recognition
    op = df['Open']
    hi = df['High']
    lo = df['Low']
    cl = df['Close']
    for pattern in getCandlePatternList(all=False):
        sig = getattr(talib, pattern)(op, hi, lo, cl)[candleIndex]
        if sig != 0:
            resultDict[pattern] = sig
    
    # Channel Breakout Indicator Signal
    cbi = ChannelBreakoutIndicator(df, ticker)
    cbi.calculate(40)
    cbSignal = cbi.getBuySell()[-1]
    if (cbSignal != ''):
        resultDict["ChannelBreakoutIndicator"] = cbSignal
        
    # Support Resistance Indicator Signal
    sri = SupportResistanceIndicator(df, 11, 5, ticker)
    sri.calculate()
    srSignal = sri.getBuySell()[-1]
    if (srSignal != ''):
        resultDict["SupportResistanceIndicator"] = srSignal

    # Collect Signals
    if (len(resultDict) != 0):
        resultDict["RSI"] = df['RSI'][candleIndex]
        tickerShortlistData[ticker]["CBI"] = cbi
        tickerShortlistData[ticker]["SRI"] = sri
        tickerShortlist[ticker] = resultDict

100%|██████████| 96/96 [04:45<00:00,  2.98s/it]


In [11]:
shortlisted = pandas.DataFrame(tickerShortlist).transpose()
shortlisted["signalCount"] = [row.count() for index, row in shortlisted.iterrows()]
shortlisted = shortlisted.sort_values(by='signalCount', ascending=False).drop(['signalCount'], axis=1)
shortlisted

,ChannelBreakoutIndicator,RSI,CDLHARAMI,CDLEVENINGDOJISTAR,CDLEVENINGSTAR,CDLENGULFING,SupportResistanceIndicator
RELIANCE,BUY,66.859021,-100,NaN,NaN,NaN,NaN
UPL,SELL,38.751811,NaN,NaN,NaN,-100,NaN
HINDUNILVR,NaN,53.933189,NaN,-100.0,-100.0,NaN,NaN
HAVELLS,SELL,44.123455,NaN,NaN,NaN,-100,NaN
ZOMATO,NaN,50.727699,NaN,NaN,NaN,-100,BUY
IOC,BUY,77.438902,NaN,NaN,NaN,NaN,NaN
NYKAA,NaN,48.193123,NaN,NaN,NaN,-100.0,NaN
NESTLEIND,NaN,59.973437,NaN,NaN,NaN,-100.0,NaN
DRREDDY,NaN,70.529265,-100.0,NaN,NaN,NaN,NaN
PIDILITIND,NaN,49.943399,NaN,NaN,NaN,-100.0,NaN


In [14]:
for ticker in shortlisted.index:
    print(tickerShortlist[ticker])
    i = tickerShortlistData[ticker]["CBI"]
    i.showIndicator(len(i.df.index)-1, 40)

{'CDLHARAMI': -100, 'ChannelBreakoutIndicator': 'BUY', 'RSI': 66.85902051518447}
0.9320660390764998 0.9861452597409583


{'CDLENGULFING': -100, 'ChannelBreakoutIndicator': 'SELL', 'RSI': 38.75181052866598}
0.3909399926959395 0.16474464579901313


{'CDLEVENINGDOJISTAR': -100, 'CDLEVENINGSTAR': -100, 'RSI': 53.93318876791824}
0.8824827601850566 0.6068847781563117


{'CDLENGULFING': -100, 'ChannelBreakoutIndicator': 'SELL', 'RSI': 44.12345456415703}
0.24778363458300504 0.9761137034503482


{'CDLENGULFING': -100, 'SupportResistanceIndicator': 'BUY', 'RSI': 50.727699066665075}
0.9402366981790162 0.8408608177269709


...

{'CDLENGULFING': -100, 'RSI': 60.47350378596976}
0 0


{'CDLHARAMI': -100, 'RSI': 51.96764621866051}
0.3290011778918873 0.9608803362427328


{'CDLHARAMI': -100, 'RSI': 66.74303008437946}
0.36033390741910926 0.6069371734677128


{'CDLHARAMI': 100, 'RSI': 68.34563812142382}
1.0 1.0


{'CDLHARAMI': -100, 'RSI': 59.27346138246996}
1.0 0.9589633823948842


In [13]:
for ticker in shortlisted.index:
    print(tickerShortlist[ticker])
    i = tickerShortlistData[ticker]["SRI"]
    i.showIndicator(len(i.df.index)-1)

{'CDLHARAMI': -100, 'ChannelBreakoutIndicator': 'BUY', 'RSI': 66.85902051518447}


{'CDLENGULFING': -100, 'ChannelBreakoutIndicator': 'SELL', 'RSI': 38.75181052866598}


{'CDLEVENINGDOJISTAR': -100, 'CDLEVENINGSTAR': -100, 'RSI': 53.93318876791824}


{'CDLENGULFING': -100, 'ChannelBreakoutIndicator': 'SELL', 'RSI': 44.12345456415703}


{'CDLENGULFING': -100, 'SupportResistanceIndicator': 'BUY', 'RSI': 50.727699066665075}


...

{'CDLENGULFING': -100, 'RSI': 60.47350378596976}


{'CDLHARAMI': -100, 'RSI': 51.96764621866051}


{'CDLHARAMI': -100, 'RSI': 66.74303008437946}


{'CDLHARAMI': 100, 'RSI': 68.34563812142382}


{'CDLHARAMI': -100, 'RSI': 59.27346138246996}
